In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy import wcs
import pickle
import wget
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def fmt(x, pos):
    a, b = "{:.2e}".format(x).split("e")
    b = int(b)
    return r"${} \times 10^{{{}}}$".format(a, b)

In [ ]:
pr3_wo_CMB_Wiener = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/zero_lvl_rm_pr3_353_wo_CMB_Wiener_reso_4.94arcmin_nside_2048.fits",
    field=None,
)
d1_bpi = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/zero_lvl_rm_d1_bpi_353_reso_4.94arcmin_nside_2048.fits",
    field=None,
)
d9_bpi = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/zero_lvl_rm_d9_bpi_353_reso_4.94arcmin_nside_2048.fits",
    field=None,
)
d12_bpi = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/zero_lvl_rm_d12_bpi_353_reso_4.94arcmin_nside_2048.fits",
    field=None,
)

In [ ]:
pr3_wo_CMB_Wiener_res_change = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/pr3_353_wo_CMB_Wiener_reso_30arcmin_nside_2048.fits",
    field=None,
)
d1_bpi_res_change = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/d1_bpi_353_reso_30arcmin_nside_2048.fits",
    field=None,
)
d9_bpi_res_change = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/d9_bpi_353_reso_30arcmin_nside_2048.fits",
    field=None,
)
d12_bpi_res_change = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/d12_bpi_353_reso_30arcmin_nside_2048.fits",
    field=None,
)

# 1] Create patch

In [ ]:
def patch_creation(n_pix, delta_pix, pos_long, pos_lat):
    w = wcs.WCS(naxis=2)  # créer un objet de taille 2D
    w.wcs.crpix = [n_pix / 2, n_pix / 2]
    w.wcs.cdelt = np.array([-delta_pix, delta_pix])
    w.wcs.crval = [pos_long, pos_lat]
    w.wcs.ctype = ["GLON-TAN", "GLAT-TAN"]
    patch = np.zeros((n_pix, n_pix))
    return (w, patch)

In [ ]:
def fill_patch(w, patch, map_fill):
    patch_index = np.indices(np.shape(patch))
    lon, lat = w.wcs_pix2world(patch_index[1], patch_index[0], 0)
    get_pix_sky = hp.ang2pix(hp.get_nside(map_fill), lon, lat, lonlat=True)
    all_pix_values = map_fill[get_pix_sky]
    filled_patch = np.reshape(all_pix_values, np.shape(patch))
    return filled_patch

In [ ]:
lon = 180
lat = -10

In [ ]:
delta_pix = 1.7 / 60
size = float(16.7)
n_pix = int(size / delta_pix)
(w, patch) = patch_creation(n_pix, delta_pix, lon, lat)
filled_patch = fill_patch(w, patch, pr3_wo_CMB_Wiener[0])

In [ ]:
def w_object(n_pix, delta_pix, pos_long, pos_lat):
    w = wcs.WCS(naxis=2)  # créer un objet de taille 2D
    w.wcs.crpix = [n_pix / 2, n_pix / 2]
    w.wcs.cdelt = np.array([-delta_pix, delta_pix])
    w.wcs.crval = [pos_long, pos_lat]
    w.wcs.ctype = ["GLON-TAN", "GLAT-TAN"]
    return w

## Intensity close to the galactic plane

In [ ]:
lon = 180
lat = -10

In [ ]:
w = w_object(n_pix, delta_pix, lon, lat)

In [ ]:
delta_pix = 1.7 / 60  # définition de la taille d'un pixel en degré
size = float(16.7)  # en degré
n_pix = int(size / delta_pix)
(w, patch) = patch_creation(n_pix, delta_pix, lon, lat)
I_pr3_wo_CMB_Wiener_gal_plane = fill_patch(w, patch, pr3_wo_CMB_Wiener[0])
I_d1_bpi_gal_plane = fill_patch(w, patch, d1_bpi[0])
I_d9_bpi_gal_plane = fill_patch(w, patch, d9_bpi[0])
I_d12_bpi_gal_plane = fill_patch(w, patch, d12_bpi[0])

In [ ]:
fig = plt.figure(figsize=(9, 7), dpi=300)
fig1 = plt.subplot(221, projection=w)
plt.imshow(
    I_pr3_wo_CMB_Wiener_gal_plane,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=2.16e3,
    vmax=2.8e4,
)
xlon = fig1.coords["glon"]
ylat = fig1.coords["glat"]
xlon.set_axislabel(" ")
ylat.set_axislabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(a)")
# plt.colorbar()

fig2 = plt.subplot(222, projection=w)
plt.imshow(I_d1_bpi_gal_plane, cmap="YlGnBu_r", origin="lower", vmin=2.16e3, vmax=2.8e4)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(b)")
# plt.colorbar()

fig3 = plt.subplot(223, projection=w)
plt.imshow(I_d9_bpi_gal_plane, cmap="YlGnBu_r", origin="lower", vmin=2.16e3, vmax=2.8e4)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(c)")
# plt.colorbar()

fig4 = plt.subplot(224, projection=w)
axcolor = plt.imshow(
    I_d12_bpi_gal_plane, cmap="YlGnBu_r", origin="lower", vmin=2.16e3, vmax=2.8e4
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(d)")

axlist = [fig1, fig2, fig3, fig4]
plt.colorbar(
    axcolor, ax=axlist, label=r"$\mu K_{CMB}$", format=ticker.FuncFormatter(fmt)
)
plt.show()
plt.savefig("../figures/gal_plane_non_smooth_wo_zero_lvl.png")

## Intensity in Bicep Keck 

In [ ]:
lon = 318
lat = -61

In [ ]:
w = w_object(n_pix, delta_pix, lon, lat)

In [ ]:
delta_pix = 1.7 / 60  # définition de la taille d'un pixel en degré
size = float(16.7)  # en degré
n_pix = int(size / delta_pix)
(w, patch) = patch_creation(n_pix, delta_pix, lon, lat)
I_pr3_wo_CMB_Wiener_gal_plane_res_change = fill_patch(
    w, patch, pr3_wo_CMB_Wiener_res_change[0]
)
I_d9_bpi_gal_plane_res_change = fill_patch(w, patch, d9_bpi_res_change[0])
I_d1_bpi_gal_plane_res_change = fill_patch(w, patch, d1_bpi_res_change[0])
I_d12_bpi_gal_plane_res_change = fill_patch(w, patch, d12_bpi_res_change[0])

### Determine the min and max for the colorscale

In [ ]:
print(np.percentile(I_pr3_wo_CMB_Wiener_gal_plane_res_change, 2))
print(np.percentile(I_pr3_wo_CMB_Wiener_gal_plane_res_change, 98))

print(np.percentile(I_d9_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d9_bpi_gal_plane_res_change, 98))

print(np.percentile(I_d1_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d1_bpi_gal_plane_res_change, 98))

print(np.percentile(I_d12_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d12_bpi_gal_plane_res_change, 98))

In [ ]:
fig = plt.figure(figsize=(9, 7), dpi=300)
fig1 = plt.subplot(221, projection=w)
plt.imshow(
    I_pr3_wo_CMB_Wiener_gal_plane_res_change,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=480,
    vmax=610,
)
xlon = fig1.coords["glon"]
ylat = fig1.coords["glat"]
xlon.set_axislabel(" ")
ylat.set_axislabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(a)")
# plt.colorbar()

fig2 = plt.subplot(222, projection=w)
plt.imshow(
    I_d1_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=70, vmax=180
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(b)")
# plt.colorbar()

fig3 = plt.subplot(223, projection=w)
plt.imshow(
    I_d9_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=475, vmax=660
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(c)")
# plt.colorbar()

fig4 = plt.subplot(224, projection=w)
axcolor = plt.imshow(
    I_d12_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=200, vmax=320
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(d)")

axlist = [fig1, fig2, fig3, fig4]
plt.colorbar(axcolor, ax=axlist, label=r"$\mu K_{CMB}$")
plt.show()
plt.savefig("../figures/BK_smooth_30'_wo_zero_lvl.png")

## Polarized intensity close to the galactic plane

In [ ]:
lon = 180
lat = -10

In [ ]:
w = w_object(n_pix, delta_pix, lon, lat)

In [ ]:
delta_pix = 1.7 / 60  # définition de la taille d'un pixel en degré
size = float(16.7)  # en degré
n_pix = int(size / delta_pix)
(w, patch) = patch_creation(n_pix, delta_pix, lon, lat)
I_pol_pr3_wo_CMB_Wiener_gal_plane_res_change = fill_patch(
    w,
    patch,
    np.sqrt(
        pr3_wo_CMB_Wiener_res_change[1] ** 2 + pr3_wo_CMB_Wiener_res_change[2] ** 2
    ),
)
I_pol_d9_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d9_bpi_res_change[1] ** 2 + d9_bpi_res_change[2] ** 2)
)
I_pol_d1_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d1_bpi_res_change[1] ** 2 + d1_bpi_res_change[2] ** 2)
)
I_pol_d12_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d12_bpi_res_change[1] ** 2 + d12_bpi_res_change[2] ** 2)
)

In [ ]:
print(np.percentile(I_pr3_wo_CMB_Wiener_gal_plane_res_change, 2))
print(np.percentile(I_pr3_wo_CMB_Wiener_gal_plane_res_change, 98))

print(np.percentile(I_d9_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d9_bpi_gal_plane_res_change, 98))

print(np.percentile(I_d1_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d1_bpi_gal_plane_res_change, 98))

print(np.percentile(I_d12_bpi_gal_plane_res_change, 2))
print(np.percentile(I_d12_bpi_gal_plane_res_change, 98))

In [ ]:
fig = plt.figure(figsize=(9, 7), dpi=300)
fig1 = plt.subplot(221, projection=w)
plt.imshow(
    I_pol_pr3_wo_CMB_Wiener_gal_plane_res_change,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=0,
    vmax=800,
)
xlon = fig1.coords["glon"]
ylat = fig1.coords["glat"]
xlon.set_axislabel(" ")
ylat.set_axislabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(a)")
# plt.colorbar()

fig2 = plt.subplot(222, projection=w)
plt.imshow(
    I_pol_d1_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=0, vmax=800
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(b)")
# plt.colorbar()

fig3 = plt.subplot(223, projection=w)
plt.imshow(
    I_pol_d9_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=0, vmax=800
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(c)")
# plt.colorbar()

fig4 = plt.subplot(224, projection=w)
axcolor = plt.imshow(
    I_pol_d12_bpi_gal_plane_res_change,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=0,
    vmax=800,
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(d)")

axlist = [fig1, fig2, fig3, fig4]
plt.colorbar(axcolor, ax=axlist, label=r"$\mu K_{CMB}$")
plt.show()
plt.savefig("../figures/pol_gal_plane_smooth_30'.png")

## Polarized intensity in Bicep Keck field

In [ ]:
lon = 318
lat = -61

In [ ]:
w = w_object(n_pix, delta_pix, lon, lat)

In [ ]:
delta_pix = 1.7 / 60  # définition de la taille d'un pixel en degré
size = float(16.7)  # en degré
n_pix = int(size / delta_pix)
(w, patch) = patch_creation(n_pix, delta_pix, lon, lat)
I_pol_pr3_wo_CMB_Wiener_gal_plane_res_change = fill_patch(
    w,
    patch,
    np.sqrt(
        pr3_wo_CMB_Wiener_res_change[1] ** 2 + pr3_wo_CMB_Wiener_res_change[2] ** 2
    ),
)
I_pol_d9_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d9_bpi_res_change[1] ** 2 + d9_bpi_res_change[2] ** 2)
)
I_pol_d1_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d1_bpi_res_change[1] ** 2 + d1_bpi_res_change[2] ** 2)
)
I_pol_d12_bpi_gal_plane_res_change = fill_patch(
    w, patch, np.sqrt(d12_bpi_res_change[1] ** 2 + d12_bpi_res_change[2] ** 2)
)

In [ ]:
fig = plt.figure(figsize=(9, 7), dpi=300)
fig1 = plt.subplot(221, projection=w)
plt.imshow(
    I_pol_pr3_wo_CMB_Wiener_gal_plane_res_change,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=10,
    vmax=45,
)
xlon = fig1.coords["glon"]
ylat = fig1.coords["glat"]
xlon.set_axislabel(" ")
ylat.set_axislabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(a)")
# plt.colorbar()

fig2 = plt.subplot(222, projection=w)
plt.imshow(
    I_pol_d1_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=10, vmax=45
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(b)")
# plt.colorbar()

fig3 = plt.subplot(223, projection=w)
plt.imshow(
    I_pol_d9_bpi_gal_plane_res_change, cmap="YlGnBu_r", origin="lower", vmin=10, vmax=45
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(c)")
# plt.colorbar()

fig4 = plt.subplot(224, projection=w)
axcolor = plt.imshow(
    I_pol_d12_bpi_gal_plane_res_change,
    cmap="YlGnBu_r",
    origin="lower",
    vmin=10,
    vmax=45,
)
plt.xlabel(" ")
plt.ylabel(" ")
plt.grid(linestyle="-", linewidth=1, color="white", alpha=0.2)
plt.title("(d)")

axlist = [fig1, fig2, fig3, fig4]
plt.colorbar(axcolor, ax=axlist, label=r"$\mu K_{CMB}$")
plt.show()
plt.savefig("../figures/pol_BK_smooth_30'.png")

## Mask

In [ ]:
mask = hp.read_map("./data/mask_zero_lvl.fits")
pr3_wo_CMB_Wiener = hp.read_map(
    "/global/cfs/cdirs/cmb/www/pysm-methods-paper/map_based_dust_validation/pr3_353_wo_CMB_Wiener_reso_4.94arcmin_nside_2048.fits",
    field=None,
)
fig0 = plt.figure(figsize=(10, 4), dpi=700)
alpha_mask = np.copy(mask)
alpha_mask[mask == 0] = 0.3
# hp.projview(pr3_wo_CMB_Wiener[0], rot = [0, 90], coord=["G"], cmap=plt.cm.Spectral_r, alpha = alpha_mask, min = 0, max = 1500, title = "", unit = r"$\mu K_{CMB}$", projection_type="polar", graticule = True, graticule_labels = True)
hp.orthview(
    pr3_wo_CMB_Wiener[0],
    rot=[0, 90],
    coord="G",
    cmap=plt.cm.YlGnBu_r,
    alpha=alpha_mask,
    min=0,
    max=1500,
    title="",
    unit=r"$\mu K_{CMB}$",
    fig=fig0.number,
)
hp.graticule(ls="-", lw=0.2, c="k")
plt.savefig("../figures/mask_intxPR3_zero_lvl.png")